In [1]:
from pkg.constraint_action import *
from pkg.constraint_object import *
from pkg.etasl import *
from pkg.geometry import *
from pkg.global_config import *
from pkg.joint_utils import *
from pkg.ros_rviz import *
from pkg.ur10 import URDF_PATH, JOINT_NAMES, LINK_NAMES, ZERO_JOINT_POSE, get_geometry_items_dict
from urdf_parser_py.urdf import URDF

# Initialize

In [2]:
urdf_path = URDF_PATH
urdf_content = URDF.from_xml_file(urdf_path)
joint_names = JOINT_NAMES
link_names = LINK_NAMES

# prepare ros
pub, joints, rate = get_publisher(joint_names)

Please create a subscriber to the marker
publication OK
published: [0, -1.5707963267948966, 0, -1.5707963267948966, 0, 0]


# Prepare collision items

In [3]:
# prepare joints
geometry_items_dict = get_geometry_items_dict(urdf_content, color=(0,1,0,0.5))

gbox = GeoBox((0.5,-0.2,0.050), (0.1,0.1,0.1), name="box_1", link_name="base_link", urdf_content=urdf_content, color=(0.8,0.8,0.8,1))
gbox_floor = GeoBox((0,0,-0.005), (3,3,0.01), name="floor", link_name="base_link", urdf_content=urdf_content, color=(0.6,0.6,0.6,1))
gbox_wall = GeoBox((0.7,0.0,0.2), (0.8,0.05,0.4), name="wall", link_name="base_link", urdf_content=urdf_content, color=(0.4,0.3,0.0,1))
gbox_stepper = GeoBox((0.4,0.4,0.15), (0.3,0.3,0.3), name="stepper", link_name="base_link", urdf_content=urdf_content, color=(0.4,0.3,0.0,1))
gbox_goal = GeoBox((0.4,0.4,0.3), (0.2,0.2,1e-3), name="platform", link_name="base_link", urdf_content=urdf_content, color=(0.8,0.0,0.0,1),
                  collision=False)
geometry_items_dict["base_link"] += [gbox]
geometry_items_dict["base_link"] += [gbox_floor]
geometry_items_dict["base_link"] += [gbox_wall]
geometry_items_dict["base_link"] += [gbox_stepper]
geometry_items_dict["base_link"] += [gbox_goal]
box_action = BoxAction(gbox)
vtool = VacuumTool([0,0,0], [0,0,1],"tool0", urdf_content, geometry_items_dict=geometry_items_dict)
pointer_plane = GeoPointer(direction=[0,0,1], _object = gbox_goal)

# prepare visualization markers
marker_list = set_markers(geometry_items_dict, joints, joint_names, link_names, urdf_content)

Please create a subscriber to the marker
publication OK
publication OK
publication OK
publication OK
publication OK
publication OK
publication OK
publication OK
publication OK
publication OK
publication OK
publication OK


# Set simulation

In [4]:
set_simulation_config(joint_names = JOINT_NAMES, link_names = LINK_NAMES, 
                      urdf_content = urdf_content, urdf_path = urdf_path,
                      geometry_items_dict=geometry_items_dict)

# Go to grasp

In [5]:
e = set_simulate(initial_jpos=ZERO_JOINT_POSE,
                 additional_constraints=vtool.make_constraints(box_action, "top"))

# show_motion

In [6]:
show_motion(e.POS, marker_list, pub, joints)

# activate grasp

In [7]:
vtool.activate(action_obj=box_action, joint_dict_last=e.joint_dict_last)

In [8]:
e = set_simulate(initial_jpos=e.POS[-1],
                 additional_constraints=make_joint_constraints(e, joint_names=joint_names), 
                 inp_lbl=['target_%s'%jname for jname in joint_names], inp=list(ZERO_JOINT_POSE))
show_motion(e.POS, marker_list, pub, joints)

# Go to put

In [9]:
e = set_simulate(initial_jpos=e.POS[-1],
                 additional_constraints=box_action.make_dir_point_constraints("bottom", pointer_plane))
show_motion(e.POS, marker_list, pub, joints)

# deactivate grasp

In [10]:
vtool.deactivate(action_obj=box_action, joint_dict_last=e.joint_dict_last, to_link='base_link')

In [11]:
e = set_simulate(initial_jpos=e.POS[-1],
                 additional_constraints=make_joint_constraints(e, joint_names=joint_names), 
                 inp_lbl=['target_%s'%jname for jname in joint_names], inp=list(ZERO_JOINT_POSE))
show_motion(e.POS, marker_list, pub, joints)

# delete all

In [31]:
for mkr in marker_list:
    mkr.delete()

# Plotting (using the Bokeh library to provide interactive plots)

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
from etasl_py.bokehplots import plotv

output_notebook()
print("positions : ")
plotv(e.TIME,e.POS,e.POS_LBL)
print("velocities : ")
plotv(e.TIME,e.VEL,e.POS_LBL)
print("outputs : ")
plotv(e.TIME,e.OUTP,e.OUTP_LBL)

In [21]:
!urdf_to_graphiz $(rospack find etasl_py_examples)/robots/ur10_robot.urdf
from graphviz import Source
Source.from_file('ur10.gv')

/bin/sh: 1: urdf_to_graphiz: not found
